# Eff. matching to GEN particle


   * [Efficiency vs $\eta$](#Effinciency-vs-$\eta$)
   * [Efficiency vs $p_{T}$](#Efficiency-vs-$p_{T}$)
   * [Turn-on curves](#Turn-on-curves)


In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

# ROOT.enableJSVis()
ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [3]:
# %load drawingTools.py
import ROOT
import math
import uuid
import pandas as pd

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 12)
stuff = []
f_idx = 0

ROOT.gStyle.SetOptTitle(False)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.13)

# ROOT.gStyle.SetCanvasBorderMode(0)
# ROOT.gStyle.SetCanvasColor(0)


def DrawPrelimLabel(canvas):
    canvas.cd()
    tex = ROOT.TLatex()
    global stuff
    stuff.append(tex)
    tex.SetTextSize(0.03)
    tex.DrawLatexNDC(0.13,0.91,"#scale[1.5]{CMS Simulation Preliminary }")
    tex.Draw("same");
    return


# void DrawLumiLabel(TCanvas* c, TString Lumi = "35.9")
# {
#   c->cd();

#   TLatex tex;
#   tex.SetTextSize(0.035);
#   TString toDisplay = Lumi + " fb^{-1} (13 TeV)";
#   tex.DrawLatexNDC(0.66,0.91,toDisplay.Data());
#   tex.Draw("same");

#   return;
# }


def SaveCanvas(canvas, name):
    canvas.cd()
    canvas.SaveAs(name+'.pdf')


# void SaveCanvas(TCanvas* c, TString PlotName = "myPlotName")
# {
#   c->cd();
#   c->SaveAs(PlotName + ".pdf");
#   c->SaveAs(PlotName + ".root");

#   return;
# }



def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def newCanvas(name=None, title=None, height=600, width=800, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title, width, height
    canvas = ROOT.TCanvas(name, title, width, height)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


def draw(histograms,
         labels,
         options='',
         norm=False,
         logy=False,
         min_y=None,
         max_y=None,
         min_x=None,
         max_x=None,
         text=None,
         y_axis_label=None,
         x_axis_label=None,
         v_lines=None,
         h_lines=None,
         do_stats=False,
         do_profile=False,
         do_ratio=False,
         do_write=False,
         write_name=None):
    global colors
    global stuff
    global p_idx

    # 0 - determine kind and # of histos
    n_histos = len(histograms)
    if n_histos == 0:
        print '[draw]**ERROR: empy histo list'
        return

    histo_class = histograms[0].ClassName()

    # 1 - check argument consisntency
    if 'TH2' not in histo_class and do_profile:
        print '[draw]**ERROR: do_profile set for histo of class: {}'.format(histo_class)
        return

    # TH1 histograms are overlayed
    # TH2 histograms are drawn side by side in the same canvas
    overlay = True
    if n_histos > 1:
        if 'TH2' in histo_class or 'TH3' in histo_class:
            overlay = False

    # 2 - determine canvas properties
    xdiv = 0
    ydiv = 0
    c_width = 800
    c_height = 600
    # do_ratio = False
    if do_write:
        c_width = 600

    if do_ratio:
        c_height = 800

    if not overlay:
        xdiv = 2
        c_width = 1000.
        ydiv = math.ceil(float(n_histos)/2)
        c_height = 500*ydiv
    canvas = newCanvas(name=None,
                       title=None,
                       height=int(c_height),
                       width=int(c_width),
                       xdiv=int(xdiv),
                       ydiv=int(ydiv))
    if overlay:
        if not do_write:
            canvas.SetRightMargin(0.30)


    canvas.cd()
    leg = getLegend()
    if do_write:
        leg = getLegend(0.6, 0.7, 0.86, 0.8)

    drawn_histos = []
    # drawn_histos = histograms
    for hidx, hist in enumerate(histograms):
        opt = options
        if 'TGraph' not in histo_class:
            hist.SetStats(do_stats)
            hist.SetTitle("")
        else:
            opt = 'P'+options
            hist.SetMarkerSize(3)
            hist.SetMarkerStyle(7)
        if overlay:
            hist.SetLineColor(colors[hidx])
            if 'TGraph' in histo_class:
                hist.SetMarkerColor(colors[hidx])
#                 hist.SetMarkerSize(3)
#                 hist.SetMarkerStyle(7)

            if hidx:
                opt = 'same,'+opt
            else:
                if 'TGraph' in histo_class:
                    opt = opt+'A'
        else:
            canvas.cd(hidx+1)
        # print hidx, opt
        d_hist = hist
        if norm:
            d_hist = hist.DrawNormalized(opt, 1.)
        else:
            hist.Draw(opt)

        if do_profile:
            profname = d_hist.GetName()+'_prof_'+str(p_idx)
            p_idx += 1
            firstbin = 1
            lastbin = -1
            prof = d_hist.ProfileX(profname, firstbin, lastbin, 's')
            prof.SetMarkerColor(2)
            prof.SetLineColor(2)
            prof.Draw('same')

        if overlay:
            if 'TGraph' not in histo_class:
                leg.AddEntry(hist, labels[hidx], 'l')
            else:
                leg.AddEntry(hist, labels[hidx], 'P')
        else:
            if text:
                newtext = '{}: {}'.format(labels[hidx], text)
                rtext = getText(newtext, 0.15, 0.85)
                rtext.Draw('same')
        drawn_histos.append(d_hist)

    if do_ratio:
        h_ratio = ROOT.TRatioPlot(drawn_histos[0], drawn_histos[1])
        stuff.append(h_ratio)
        h_ratio.Draw()

        pad = canvas.cd(1)
        h_ratio.GetUpperPad().SetBottomMargin(0)
        h_ratio.GetUpperPad().SetRightMargin(0.3)
        if logy:
            h_ratio.GetUpperPad().SetLogy()
        pad.Update()
        pad = canvas.cd(2)
        h_ratio.GetLowerPad().SetTopMargin(0)
        h_ratio.GetLowerPad().SetRightMargin(0.3)
        pad.Update()
        # hratio.GetLowerRefGraph().GetYaxis().SetRangeUser(-6, 6)

    canvas.Update()

    # we now set the axis properties
    max_value = max_y
    min_value = min_y

    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in drawn_histos])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in drawn_histos])*1.2
    # print min_value, max_value

    for hist in drawn_histos:
        hist.GetYaxis().SetRangeUser(min_value, max_value)
        if y_axis_label:
            hist.GetYaxis().SetTitle(y_axis_label)
        if x_axis_label:
            hist.GetXaxis().SetTitle(x_axis_label)
        if min_x and max_x:
            hist.GetXaxis().SetRangeUser(min_x, max_x)


    canvas.Draw()

    # we draw additional stuff if needed
    if overlay:
        canvas.cd()
        if len(labels) > 1:
            leg.Draw()
        if text:
            rtext = getText(text, 0.15, 0.85)
            rtext.Draw("same")

    pad_range = range(0, 1)
    if not overlay:
        pad_range = range(1, len(histograms)+1)

    for pad_id in pad_range:
        canvas.cd(pad_id)
        if v_lines:
            for v_line_x in v_lines:
                aline = ROOT.TLine(v_line_x, ROOT.gPad.GetUymin(), v_line_x,  ROOT.gPad.GetUymax())
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if h_lines:
            for h_line_y in h_lines:
                aline = ROOT.TLine(ROOT.gPad.GetUxmin(), h_line_y, ROOT.gPad.GetUxmax(),  h_line_y)
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if logy:
            if not do_ratio:
                ROOT.gPad.SetLogy()

        ROOT.gPad.Update()



    canvas.Update()
    if(do_write):
        DrawPrelimLabel(canvas)

    canvas.Draw()
    if(do_write):
        SaveCanvas(canvas, write_name)

    return canvas


files = {}
file_keys = {}


class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys


class Sample():
    def __init__(self, name, label, version=None, type=None):
        self.name = name
        self.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        self.histo_filename = '../plots1/histos_{}{}.root'.format(self.name, version)
        self.histo_file = None
        self.type = type

    def __repr__(self):
        return '<{} {}, {}>'.format(self.__class__.__name__, self.histo_filename, self.type)

    def open_file(self):
        self.histo_file = ROOT.TFile(self.histo_filename, 'r')

# sample_names = ['ele_flat2to100_PU0',
#                 'ele_flat2to100_PU200',
#                 'photonPt35_PU0',
#                 'photonPt35_PU200']


class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None

    def get(self, debug=False):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel is None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            if debug:
                print '-- HProxy:Get: {}'.format(name)
            self.instance = self.classtype(name, self.root_file, debug)
        return self.instance


class HPlot:
    def __init__(self, samples, tp_sets, tp_selections, gen_selections):
        self.tp_sets = tp_sets
        # self.tp_selections = tp_selections
        # self.gen_selections = gen_selections
        self.pus = []
        self.labels_dict = {}

        for sample in samples:
            self.pus.append(sample.label)
            self.labels_dict[sample.type] = sample.type

        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])

        self.labels_dict.update(tp_sets)
        self.labels_dict.update(tp_selections)
        self.labels_dict.update(gen_selections)
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

    def cache_histo(self,
                    classtype,
                    samples,
                    pus,
                    tps,
                    tp_sels,
                    gen_sels):
        if gen_sels is None:
            gen_sels = [None]

        for sample in samples:
            print sample
            for tp in tps:
                for tp_sel in tp_sels:
                    for gen_sel in gen_sels:
                        print sample, tp, tp_sel, gen_sel
                        self.data = self.data.append({'sample': sample.type,
                                                      'pu': sample.label,
                                                      'tp': tp,
                                                      'tp_sel': tp_sel,
                                                      'gen_sel': gen_sel,
                                                      'classtype': classtype,
                                                      'histo': HProxy(classtype, tp, tp_sel, gen_sel, sample.histo_file)},
                                                     ignore_index=True)

    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None,
                  debug=False):
        histo = None
        labels = []
        text = ''

        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += ' & (gen_sel == @gen_sel)'
        else:
            query += ' & (gen_sel.isnull())'
        if sample is not None:
            query += '& (sample == @sample)'

        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)
            return None, None, None
        if debug:
            print histo_df

        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])
#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        # print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get(debug) for his in histo_df['histo'].values]
        return histo, labels, text


In [5]:
# %load samples.py
import ROOT
import pandas as pd

version = 'v103t'
version_V8 = 'v108t'

version_V9 = 'v113'

# -------------------------------------------------------------------------

samples_ele = [
#     Sample('ele_flat2to100_PU0_eg', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200_eg', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
#     Sample('ele_flat2to100_PU0_eg', 'PU0', version, 'V8'),
    Sample('ele_flat2to100_PU200_eg', 'PU200', version_V8, 'V8')
    ]


samples_ele_V9 = [
    Sample('ele_flat2to100_PU0_eg', 'PU0', version_V9, 'ele-V9'),
    Sample('ele_flat2to100_PU200_eg', 'PU200', version_V9, 'ele-V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0_tracks', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200_tracks', 'PU200', version, 'photon')
    ]

samples_photons_V9 = [
    Sample('photon_flat8to150_PU0_tracks', 'PU0', version_V9, 'photon-V9'),
    Sample('photon_flat8to150_PU200_tracks', 'PU200', version_V9, 'photon-V9')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0_tracks', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200_tracks', 'PU200', version, 'mb')
    ]




In [9]:
do_eff = True

In [12]:
# %load settings.py
# === samples =====================================================
import python.selections as selections
import python.collections as collections

import pprint


samples = []

# samples += samples_nugunrates
# samples += samples_nugunrates_V8
samples += samples_ele_V9

for smp in samples:
    smp.open_file()


sample = 'ele-V9'

do_rate = False
do_eff = True

# === TP ==========================================================
tps = [
       'EG',
       'TkEle',
       'TkEleEL',
       'TkEleBRL',
       'TkEleELBRL'
      ]

# === Load the Histo Primitives ====================================
histo_primitives = pd.DataFrame()

if do_rate:
    from python.plotters import rate_plotters, eg_rate_plotters

    for plotter in rate_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
    for plotter in eg_rate_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
if do_eff:
    from python.plotters import eg_genmatched_plotters, track_genmatched_plotters
    for plotter in eg_genmatched_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
    for plotter in track_genmatched_plotters:
        histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)
        

# print histo_primitives.data.unique()
# === TP selections ================================================
tp_select = {}

for tp in tps:
    tp_select[tp] = histo_primitives[histo_primitives.data == tp].data_sel.unique().tolist()

# ==== GEN selections ===============================================
gen_select ={}
for tp in tps:
    gen_select[tp] = histo_primitives[histo_primitives.data == tp].gen_sel.unique().tolist()

#  ==== labels ===============================================
tp_labels = histo_primitives[['data', 'data_label']].drop_duplicates().set_index('data').T.to_dict('records')[0]
tp_selection_labels = histo_primitives[['data_sel', 'data_sel_label']].drop_duplicates().set_index('data_sel').T.to_dict('records')[0]
gen_selection_labels = histo_primitives[['gen_sel', 'gen_sel_label']].drop_duplicates().set_index('gen_sel').T.to_dict('records')[0]



import pprint
pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
pp.pprint(gen_select)


# print selections.eg_rate_selections


--- TPs: 
['EG', 'TkEle', 'TkEleEL', 'TkEleBRL', 'TkEleELBRL']
--- TP selections:
{   'EG': [   'EGq2',
              'EGq2Pt10',
              'EGq2Pt15',
              'EGq2Pt20',
              'EGq2Pt25',
              'EGq2Pt30',
              'EGq2Pt40',
              'EGq3',
              'EGq3Pt10',
              'EGq3Pt15',
              'EGq3Pt20',
              'EGq3Pt25',
              'EGq3Pt30',
              'EGq3Pt40',
              'EGq4',
              'EGq4Pt10',
              'EGq4Pt15',
              'EGq4Pt20',
              'EGq4Pt25',
              'EGq4Pt30',
              'EGq4Pt40',
              'EGq5',
              'EGq5Pt10',
              'EGq5Pt15',
              'EGq5Pt20',
              'EGq5Pt25',
              'EGq5Pt30',
              'EGq5Pt40'],
    'TkEle': [   'EGq2',
                 'EGq2Pt10',
                 'EGq2Pt15',
                 'EGq2Pt20',
                 'EGq2Pt25',
                 'EGq2Pt30',
                 'EGq2Pt40',
      

/Users/cerminar/.virtualenvs/HGCTPGPerformance/lib/python2.7/site-packages/ipykernel_launcher.py:65: UserWarning: DataFrame columns are not unique, some columns will be omitted.


In [14]:
for tp in tps:
    print 'TP: {}'.format(tp)
    print histo_primitives[histo_primitives.data == tp].data_sel.unique()
    print histo_primitives[histo_primitives.data == tp].gen_sel.unique()

TP: EG
['EGq2' 'EGq2Pt10' 'EGq2Pt15' 'EGq2Pt20' 'EGq2Pt25' 'EGq2Pt30' 'EGq2Pt40'
 'EGq3' 'EGq3Pt10' 'EGq3Pt15' 'EGq3Pt20' 'EGq3Pt25' 'EGq3Pt30' 'EGq3Pt40'
 'EGq4' 'EGq4Pt10' 'EGq4Pt15' 'EGq4Pt20' 'EGq4Pt25' 'EGq4Pt30' 'EGq4Pt40'
 'EGq5' 'EGq5Pt10' 'EGq5Pt15' 'EGq5Pt20' 'EGq5Pt25' 'EGq5Pt30' 'EGq5Pt40']
['GEN' 'GENPt15' 'GENPt20' 'GENPt30' 'GENPt40' 'GENEtaD' 'GENEtaBC'
 'GENEtaBCD']
TP: TkEle
['EGq2' 'EGq2Pt10' 'EGq2Pt15' 'EGq2Pt20' 'EGq2Pt25' 'EGq2Pt30' 'EGq2Pt40'
 'EGq3' 'EGq3Pt10' 'EGq3Pt15' 'EGq3Pt20' 'EGq3Pt25' 'EGq3Pt30' 'EGq3Pt40'
 'EGq4' 'EGq4Pt10' 'EGq4Pt15' 'EGq4Pt20' 'EGq4Pt25' 'EGq4Pt30' 'EGq4Pt40'
 'EGq5' 'EGq5Pt10' 'EGq5Pt15' 'EGq5Pt20' 'EGq5Pt25' 'EGq5Pt30' 'EGq5Pt40'
 'EGq2Iso0p2' 'EGq2Iso0p1' 'EGq3Iso0p2' 'EGq3Iso0p1' 'EGq4Iso0p2'
 'EGq4Iso0p1' 'EGq5Iso0p2' 'EGq5Iso0p1']
['GEN' 'GENPt15' 'GENPt20' 'GENPt30' 'GENPt40' 'GENEtaBC' 'GENEtaBCD']
TP: TkEleEL
['EGq2' 'EGq2Pt10' 'EGq2Pt15' 'EGq2Pt20' 'EGq2Pt25' 'EGq2Pt30' 'EGq2Pt40'
 'EGq3' 'EGq3Pt10' 'EGq3Pt15' 'EGq3Pt20' 'EG

In [15]:
tp_labels

{'EG': 'EG',
 'L1Trk': 'L1Track',
 'L1TrkEmu': 'L1Track EMU',
 'TkEle': 'TkEle',
 'TkEleBRL': 'TkEle B',
 'TkEleEL': 'TkEle ELLIPTIC',
 'TkEleELBRL': 'TkEle ELLIPTIC B'}

In [16]:
tp_selection_labels

{'EGq2': 'hwQual 2',
 'EGq2Iso0p1': 'hwQual 2, Iso0p1',
 'EGq2Iso0p2': 'hwQual 2, Iso0p2',
 'EGq2Pt10': 'hwQual 2, p_{T}^{L1}>=10GeV',
 'EGq2Pt15': 'hwQual 2, p_{T}^{L1}>=15GeV',
 'EGq2Pt20': 'hwQual 2, p_{T}^{L1}>=20GeV',
 'EGq2Pt25': 'hwQual 2, p_{T}^{L1}>=25GeV',
 'EGq2Pt30': 'hwQual 2, p_{T}^{L1}>=30GeV',
 'EGq2Pt40': 'hwQual 2, p_{T}^{L1}>=40GeV',
 'EGq3': 'hwQual 3',
 'EGq3Iso0p1': 'hwQual 3, Iso0p1',
 'EGq3Iso0p2': 'hwQual 3, Iso0p2',
 'EGq3Pt10': 'hwQual 3, p_{T}^{L1}>=10GeV',
 'EGq3Pt15': 'hwQual 3, p_{T}^{L1}>=15GeV',
 'EGq3Pt20': 'hwQual 3, p_{T}^{L1}>=20GeV',
 'EGq3Pt25': 'hwQual 3, p_{T}^{L1}>=25GeV',
 'EGq3Pt30': 'hwQual 3, p_{T}^{L1}>=30GeV',
 'EGq3Pt40': 'hwQual 3, p_{T}^{L1}>=40GeV',
 'EGq4': 'hwQual 4',
 'EGq4Iso0p1': 'hwQual 4, Iso0p1',
 'EGq4Iso0p2': 'hwQual 4, Iso0p2',
 'EGq4Pt10': 'hwQual 4, p_{T}^{L1}>=10GeV',
 'EGq4Pt15': 'hwQual 4, p_{T}^{L1}>=15GeV',
 'EGq4Pt20': 'hwQual 4, p_{T}^{L1}>=20GeV',
 'EGq4Pt25': 'hwQual 4, p_{T}^{L1}>=25GeV',
 'EGq4Pt30': 'hwQual 4,

In [17]:
gen_selection_labels

{'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENEtaF': '|#eta^{GEN}| <= 1.47',
 'GENPt15': 'p_{T}^{GEN}>=15GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV'}

In [19]:
%%time

hplot = HPlot(samples, tp_labels, tp_selection_labels, gen_selection_labels)

for tp in tps:
    hplot.cache_histo(classtype=histos.HistoSetEff, 
                      samples=samples,
                      pus=[],
                      tps=[tp],
                      tp_sels=histo_primitives[histo_primitives.data == tp].data_sel.unique(),
                      gen_sels=gen_select[tp])



<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9>
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq2 GEN
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq2 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq2 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq2 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq2 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq2 GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq2 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq2 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq2Pt10 GEN
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq2Pt10 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq2Pt10 GENPt20
<Sample ../plots1/histos_

<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq4 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq4 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq4 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq4 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq4 GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq4 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq4 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq4Pt10 GEN
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq4Pt10 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq4Pt10 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq4Pt10 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> EG EGq4Pt10 GENPt

<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq2 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq2 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq2 GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq2 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq2 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq2Pt10 GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq2Pt10 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq2Pt10 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq2Pt10 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq2Pt10 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq2Pt10 GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113

<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq4 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq4 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq4Pt10 GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq4Pt10 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq4Pt10 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq4Pt10 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq4Pt10 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq4Pt10 GENEtaD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq4Pt10 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq4Pt10 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq4Pt15 GEN
<Sample ../plots1/histos_ele_flat2to100_PU

<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq5Pt40 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> EG EGq5Pt40 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9>
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> TkEle EGq2 GEN
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> TkEle EGq2 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> TkEle EGq2 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> TkEle EGq2 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> TkEle EGq2 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> TkEle EGq2 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> TkEle EGq2 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> TkEle EGq2Pt10 GEN
<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> TkEle EGq2

...

<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9> TkEleEL EGq5Iso0p1 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9>
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2 GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2Pt10 GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2Pt10 GENPt15
<Sample ../plots1/histos_ele_flat

<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq3Pt30 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq3Pt30 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq3Pt30 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq3Pt40 GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq3Pt40 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq3Pt40 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq3Pt40 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq3Pt40 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq3Pt40 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq3Pt40 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> 

<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2Iso0p2 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2Iso0p2 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2Iso0p2 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2Iso0p2 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2Iso0p2 GENEtaBC
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2Iso0p2 GENEtaBCD
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2Iso0p1 GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2Iso0p1 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2Iso0p1 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleEL EGq2Iso0p1 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v1

<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleBRL Pt25 GENEtaF
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleBRL Pt30 GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleBRL Pt30 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleBRL Pt30 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleBRL Pt30 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleBRL Pt30 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleBRL Pt30 GENEtaF
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleBRL Pt40 GEN
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleBRL Pt40 GENPt15
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleBRL Pt40 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleBRL Pt40 GENPt30
<Sample ../plots1/

<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleELBRL Pt40 GENPt20
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleELBRL Pt40 GENPt30
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleELBRL Pt40 GENPt40
<Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9> TkEleELBRL Pt40 GENEtaF
CPU times: user 10.1 s, sys: 1.69 s, total: 11.7 s
Wall time: 11 s


In [20]:
print dir(samples[0])
samples[0].histo_file.ls()
samples

['__doc__', '__init__', '__module__', '__repr__', 'histo_file', 'histo_filename', 'label', 'name', 'open_file', 'type']


[<Sample ../plots1/histos_ele_flat2to100_PU0_eg_v113.root, ele-V9>,
 <Sample ../plots1/histos_ele_flat2to100_PU200_eg_v113.root, ele-V9>]

TFile**		../plots1/histos_ele_flat2to100_PU0_eg_v113.root	
 TFile*		../plots1/histos_ele_flat2to100_PU0_eg_v113.root	
  KEY: TDirectoryFile	EGHistos;1	EGHistos
  KEY: TDirectoryFile	EGResoHistos;1	EGResoHistos
  KEY: TDirectoryFile	GenParticleHistos;1	GenParticleHistos


In [21]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'TkEle', 'EGq4Pt20', ['GENEtaBC'], debug=False)
print hsets
hsets[0].name
hsets[0].name
dir(hsets[0].h_eff)

[<python.l1THistos.HistoSetEff instance at 0x11bf92050>]


['__doc__',
 '__init__',
 '__module__',
 'h_abseta',
 'h_energy',
 'h_eta',
 'h_fBrem',
 'h_pt',
 'h_reachedEE']

In [22]:
%%time

for smp in hplot.data['sample'].unique():
    print 'Sample: {}'.format(smp)
    for pu in hplot.data[(hplot.data['sample'] == smp)].pu.unique():
        for tp in ['EG', 'TkEleEL']:
            print 'PU: {}, TP: {}'.format(pu, tp)
            for tp_sel in hplot.data[(hplot.data['sample'] == smp) & (hplot.data.pu == pu) & (hplot.data.tp == tp)].tp_sel.unique():
                if 'Pt' not in tp_sel:
                    continue
#                 tp_sel_den = 'all'
                # print tp_sel
                tp_sel_den = tp_sel.split('Pt')[0]
                if tp_sel_den == '':
                    tp_sel_den = 'all'
                print '  {} den -> {}'.format(tp_sel, tp_sel_den) 
                for gen_sel in hplot.data[(hplot.data['sample'] == smp) & (hplot.data.pu == pu) & (hplot.data.tp == tp) & (hplot.data.tp_sel == tp_sel)].gen_sel.unique():
                    if gen_sel == 'nomatch' or 'Pt' in gen_sel:
                        continue
                    print gen_sel
                    hsetden = hplot.get_histo(histos.HistoSetEff, smp, pu, tp, tp_sel_den, gen_sel)
                    hset = hplot.get_histo(histos.HistoSetEff, smp, pu, tp, tp_sel, gen_sel)
                    hset[0][0].computeTurnOn(hsetden[0][0].h_num)


Sample: ele-V9
PU: PU0, TP: EG
  EGq2Pt10 den -> EGq2
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq2Pt15 den -> EGq2
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq2Pt20 den -> EGq2
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq2Pt25 den -> EGq2
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq2Pt30 den -> EGq2
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq2Pt40 den -> EGq2
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq3Pt10 den -> EGq3
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq3Pt15 den -> EGq3
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq3Pt20 den -> EGq3
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq3Pt25 den -> EGq3
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq3Pt30 den -> EGq3
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq3Pt40 den -> EGq3
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq4Pt10 den -> EGq4
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq4Pt15 den -> EGq4
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq4Pt20 den -> EGq4
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq4Pt25 den -> EGq4
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq4Pt30 den -> EGq4
GEN
GENEtaD
GENEtaBC
GENEtaBCD
  EGq4Pt40 den -> EGq4
GEN
GENEtaD
GENEtaBC
GENEtaB

In [23]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,ele-V9,PU0,EG,EGq2,GEN,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11bc6d5f0>
1,ele-V9,PU0,EG,EGq2,GENPt15,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11bf05d40>
2,ele-V9,PU0,EG,EGq2,GENPt20,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11bf05a70>
3,ele-V9,PU0,EG,EGq2,GENPt30,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11bcccf80>
4,ele-V9,PU0,EG,EGq2,GENPt40,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11bd72098>
5,ele-V9,PU0,EG,EGq2,GENEtaD,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11bd72050>
6,ele-V9,PU0,EG,EGq2,GENEtaBC,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11bc6df80>
7,ele-V9,PU0,EG,EGq2,GENEtaBCD,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11bc1b488>
8,ele-V9,PU0,EG,EGq2Pt10,GEN,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11bd81488>
9,ele-V9,PU0,EG,EGq2Pt10,GENPt15,python.l1THistos.HistoSetEff,<__main__.HProxy instance at 0x11bf031b8>


In [24]:
hplot.data['sample'].unique()

array(['ele-V9'], dtype=object)

In [25]:
hplot.data[hplot.data.tp == 'EG'].tp_sel.unique()

array(['EGq2', 'EGq2Pt10', 'EGq2Pt15', 'EGq2Pt20', 'EGq2Pt25', 'EGq2Pt30',
       'EGq2Pt40', 'EGq3', 'EGq3Pt10', 'EGq3Pt15', 'EGq3Pt20', 'EGq3Pt25',
       'EGq3Pt30', 'EGq3Pt40', 'EGq4', 'EGq4Pt10', 'EGq4Pt15', 'EGq4Pt20',
       'EGq4Pt25', 'EGq4Pt30', 'EGq4Pt40', 'EGq5', 'EGq5Pt10', 'EGq5Pt15',
       'EGq5Pt20', 'EGq5Pt25', 'EGq5Pt30', 'EGq5Pt40'], dtype=object)

In [26]:
# sample='ele'

In [27]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'TkEle', 'EGq4Pt25', ['GENEtaBC'], debug=False)
print hsets
hsets[0].name

[<python.l1THistos.HistoSetEff instance at 0x11c370c20>]


'TkEle_EGq4Pt25_GENEtaBC'

## Effinciency vs $\eta$

In [ ]:
dir(hplot)


In [ ]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in ['GEN']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['ele', 'V8', 'ele-V9'], ['PU200'], tp, tp_sel, gen_sel, debug=False)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
#             print [his.h_num.h_eta.GetName() for his in hsets]

In [ ]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in ['GEN']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel, debug=False)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
#             print [his.h_num.h_eta.GetName() for his in hsets]

In [ ]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.75, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
     

In [ ]:
# for tp in tps:
#     for tp_sel in tp_select[tp]:
#         for gen_sel in ['GENPt30']:
for pu in ['PU0', 'PU200']:
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'all', ['GENPt30'])
# drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.75, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
for pu in ['PU0', 'PU200']:

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'Pt2', ['GENPt30'])
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.8, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
for pu in ['PU0', 'PU200']:
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'Pt10', ['GENPt30'])
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.8, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
    

In [ ]:
for pu in ['PU0', 'PU200']:
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'St4Pt10', ['GENPt30'])
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.8, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
    

In [ ]:
for pu in ['PU0', 'PU200']:
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'St4Pt2', ['GENPt30'])
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.8, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0'], ['L1Trk', 'L1TrkEmu'], 'all', ['GENPt30'])
# drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
# for tp in tps:
for pu in ['PU0', 'PU200']:
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle', 'TkIsoEle'], ['EGq2Iso0p2'], 'GENPt20')
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
for pu in ['PU0', 'PU200']:
    for tp in ['TkEG']:
#     for tp_sel in tp_select[tp]:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, ['EGq2' ,'EGq2M2','EGq2M2s','EGq2M3','EGq2M3s', 'EGq2M4'], gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
     

In [ ]:
tp_select['TkEG']

In [ ]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in ['GENPt20']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'V8', ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
for pu in ['PU0', 'PU200']:
    for tp in ['EG', 'TkEle']:
        for tp_sel in tp_select[tp]:
            for gen_sel in ['GENPt20']:
                print tp, tp_sel
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['V8', 'V9'], pu, tp, tp_sel, gen_sel)
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
# for tp in tps:
#     for tp_sel in tp_select[tp]:
#         for gen_sel in ['GENPt20']:

for pu in ['PU0', 'PU200']:

    hsets_new = []
    labels_new = []
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq2'], 'GENPt20')
    hsets_new.append(hsets[0])
    labels_new.append("EG standalone")
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['Pt2'], 'GENPt20')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>2GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['Pt10'], 'GENPt20')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>10GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq2'], 'GENPt20')
    hsets_new.append(hsets[0])
    labels_new.append("TkEle")

    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_eta.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], 'EG', 'EGq4Pt25', 'GENPt30', debug=False)
labels = ['TP head', 'L1 menu version', 'backported']
draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.6, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
     do_write=True, write_name='egammaEndcap_EG_EGq4Pt25_GENPt30_hEffVEta')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EG','TkEleEL'], ['EGq3Pt25', 'EGq5Pt25'], 'GENPt30', debug=False)
draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.6, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
     do_write=True, write_name='egammaEndcap_TkEle_EGq4Pt30_GENPt40_hEffVEta')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], ['TkEleEL'], ['EGq4Pt25', 'EGq4Iso0p2Pt25', 'EGq4Iso0p3Pt25', 'EGq5Pt25'], 'GENPt30', debug=False)
draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.6, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EG', 'TkEle', 'TkEleEL'], 'EGq3Pt30', 'GENPt40', debug=False)
draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.6, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
     do_write=True, write_name='egammaEndcap_TkEle_EGq4Pt30_GENPt40_hEffVEta')





In [54]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEleBRL', 'TkEleELBRL'], ['Pt30'], ['GENPt40'], debug=False)
draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.6, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
     do_write=False, write_name='egammaEndcap_TkEle_EGq4Pt30_GENPt40_hEffVEta')



<ROOT.TCanvas object ("c_2d08d3") at 0x7fdd4fa93670>

In [45]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEleELBRL'], 'Pt20', 'GENPt30', debug=False)

newset = []

h_barrel = hsets[0].h_eff.h_eta.CreateGraph()

hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEleEL'], 'EGq5Pt20', 'GENPt30', debug=False)

h_endcap = hsets[0].h_eff.h_eta.CreateGraph()

newset.append(h_barrel)
newset.append(h_endcap)

newlabels = ['barrel', 'endcap']

draw(newset, newlabels, text=text, min_y=0.6, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
     do_write=True, write_name='egammaEndcap_TkEle_EGq4Pt30_GENPt40_hEffVEta')


<ROOT.TCanvas object ("c_03da92") at 0x7fdd4fa77e40>

Info in <TCanvas::Print>: pdf file egammaEndcap_TkEle_EGq4Pt30_GENPt40_hEffVEta.pdf has been created


In [34]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEleBRL', 'TkEleELBRL'], 'Pt20', 'GENPt30', debug=False)
draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.6, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
     do_write=False, write_name='egammaEndcap_TkEle_EGq4Pt30_GENPt40_hEffVEta')


<ROOT.TCanvas object ("c_8901f5") at 0x7fdd4fb44080>

## Efficiency vs $p_{T}$

In [ ]:
line_y1p0 = ROOT.TLine(0.,1., 110., 1.)
line_y0p95 = ROOT.TLine(0.,0.95, 110., 0.95)
line_y0p95.SetLineStyle(2)

In [ ]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
    for gen_sel in gen_select:
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'V8', pu, ['EG'], 'EGq2', gen_sel)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
 

In [ ]:
for pu in ['PU0', 'PU200']:
    for gen_sel in gen_select:
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'V8', pu, ['TkEle', 'TkIsoEle'], 'EGq2Iso0p2', gen_sel)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
 

In [ ]:
for pu in ['PU0']:
    for gen_sel in gen_select:
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'all', gen_sel)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
 

In [ ]:
for pu in ['PU0', 'PU200']:
    for gen_sel in gen_select:
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'Pt2', gen_sel)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
 

In [ ]:
for pu in ['PU0', 'PU200']:
    for gen_sel in gen_select:
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk', 'L1TrkEmu'], 'Pt10', gen_sel)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])
 

In [ ]:
sample = 'ele-V9'
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []
    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("EG standalone")
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3Pt20'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("EG Pt>20GeV")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['Pt2'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("L1Trk p_{T}>2GeV")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['Pt10'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("L1Trk p_{T}>10GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("TkEle")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEle")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P10'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P5'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("TkEG TkP5")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P2'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG TkP2")

    text = '{}, HWQual==3, 1.52<|{}|<=2.4'.format(pu, '#eta^{GEN}')
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_pt.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.55, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


In [ ]:
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq3M1P10', 'EGq3M1P5', 'EGq3M1P2'], 'GENEtaBC')
    hsets_new.append(hsets[2])
    labels_new.append('p_{T}^{track}>10GeV')
    hsets_new.append(hsets[1])
    labels_new.append('p_{T}^{track}>5GeV')

    hsets_new.append(hsets[0])
    labels_new.append('p_{T}^{track}>2GeV')

    #     labels_new.append("TkEG")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P5'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG TkP5")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P2'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG TkP2")

    
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_pt.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.7, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


In [ ]:
sample = 'V8'
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P10', 'EGq2M1P5S', 'EGq2M1P2S'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P5'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG TkP5")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq2M1P2'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEG TkP2")

    
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0.7, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


## Efficiency vs fBrem

In [ ]:
sample = 'ele-V9'
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3'], 'GENEtaBC')
# #     hsets[0].SetMarkerStyle(7)
#     hsets_new.append(hsets[0])
#     labels_new.append("EG standalone")
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3Pt20'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("EG Pt>20GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['St4Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk #stubs > 3")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1Trk'], ['St4Pt10'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("L1Trk p_{T}>10GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk EMU")

#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt10'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("L1Trk p_{T}>10GeV")

    
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("TkEle")


    
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_fBrem.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.02, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


In [ ]:
sample = 'ele-V9'
for pu in ['PU0', 'PU200']:
    hsets_new = []
    labels_new = []
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3'], 'GENEtaBC')
# #     hsets[0].SetMarkerStyle(7)
#     hsets_new.append(hsets[0])
#     labels_new.append("EG standalone")
    
#     hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq3Pt20'], 'GENEtaBC')
#     hsets_new.append(hsets[0])
#     labels_new.append("EG Pt>20GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt2'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>2GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['L1TrkEmu'], ['Pt10'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("L1Trk p_{T}>10GeV")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("TkEle")

    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEG'], ['EGq3M1P10'], 'GENEtaBC')
    hsets_new.append(hsets[0])
    labels_new.append("TkEle")


    
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    draw([his.h_eff.h_fBrem.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.02, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', 'EGq4Pt25', 'GENEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1],
     do_write=True, write_name='egammaEndcap_EG_EGq4Pt25_GENEtaBC_hEffvPt')
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], 'EG', 'EGq4Pt25', 'GENEtaD')
print labels
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1],
     do_write=True, write_name='egammaEndcap_EG_EGq4Pt25_GENEtaD_hEffvPt')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEle'], ['EGq4Pt20', 'EGq4Pt30'], 'GENEtaBC')
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1],
     do_write=False, write_name='egammaEndcap_TkEle_EGq4Pt20_GENEtaBC_hEffvPt')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EG', 'TkEle'], 'EGq4Pt30', 'GENEtaBC')
labels=['standalone cluster', 'track matched clusters']
draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1],
     do_write=True, write_name='egammaEndcap_TkEle_EGq4Pt30_GENEtaBC_hEffvPt')


## Turn-on curves

In [ ]:
for tp in tps:
#     if tp in ['TkEle', 'TkIsoEle', 'L1Trk']:
#         continue
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            tp_sel+='Pt30'
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel, debug=False)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['EG'], ['EGq3Pt30', 'EGq4Pt30', 'EGq5Pt30'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU200'], ['TkEleEL'], ['EGq3Pt30', 'EGq4Pt30', 'EGq5Pt30'], 'GENEtaBC', debug=False)
draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
for tp in tps:
    if tp in ['TkEG', 'TkIsoEle', 'L1Trk']:
        continue
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            tp_sel+='Pt20'
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel, debug=False)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
for tp in tps:
    if tp in ['TkEG', 'TkIsoEle', 'L1Trk']:
        continue
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            tp_sel+='Pt10'
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel, debug=True)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:

for pu in ['PU0']:
    for gen_sel in gen_select:
        tp_sel+='Pt20'
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['EG'], ['EGq2Pt20', 'EGq3Pt20'], gen_sel, debug=True)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:

for pu in ['PU0']:
    for gen_sel in gen_select:
        tp_sel+='Pt20'
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3Pt20', 'EGq4Pt20'], gen_sel, debug=True)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:

for pu in ['PU200']:
    for gen_sel in gen_select:
        tp_sel+='Pt20'
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3Pt20', 'EGq4Pt20'], gen_sel, debug=True)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:

for pu in ['PU200']:
    for gen_sel in gen_select:
        tp_sel+='Pt25'
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3Pt30', 'EGq4Pt30'], gen_sel, debug=True)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:

for pu in ['PU200']:
    for gen_sel in gen_select:
        tp_sel+='Pt30'
        hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, ['TkEle'], ['EGq3Pt30', 'EGq4Pt30'], gen_sel, debug=True)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])


In [ ]:
for tp in tps:
    for tp_sel in ['Pt20', 'EmPt20', 'Emv1Pt20', 'Pt30', 'EmPt30', 'Emv1Pt30']:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            drawSame([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
    for tp in tps:
        for tp_sel in ['Pt20', 'EmPt20', 'Emv1Pt20', 'Pt30', 'EmPt30', 'Emv1Pt30']:
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, tp_sel, ['GENEtaABC', 'GENEtaBC'])
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                drawSame([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
        for tp_sel in ['EmPt20', 'Emv1Pt20', 'EmPt30', 'Emv1Pt30']:
            for gen_sel in gen_select:
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, ['DEF', 'DEFCalib'], tp_sel, gen_sel)
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                drawSame([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
        for tp_sel in ['EmPt20', 'Emv1Pt20', 'EmPt30', 'Emv1Pt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, ['DEF', 'DEFCalib'], tp_sel, ['GENEtaABC', 'GENEtaBC'])
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            drawSame([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

## Scaling

In [ ]:
def f_yc(x, par):
    return (ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], 0) - ROOT.TMath.Exp(-par[0]*(x[0]-par[1])+par[0]*par[0]*par[2]*par[2]/2)*ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], par[0]*par[0]*par[2]*par[2])) * (par[3] - par[4]) + par[4]

tf_yc = ROOT.TF1('tf_yc', f_yc, 0, 100, 5)
tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
tf_yc.SetParNames("#lambda", "#mu", "#sigma", "Plateau", "Baseline");
tf_yc.SetLineColor(6)
tf_yc.FixParameter(4,0)

tf_yc.SetParLimits(1, 0, 50)

tf_yc.SetParLimits(3, 0.9, 1)
tf_yc.SetParLimits(0, 0, 1)
tf_yc.SetParLimits(2, 0.1, 1)

c = newCanvas()
tf_yc.Draw()
c.Draw()


In [ ]:
sample

In [ ]:
hplot.data[(hplot.data['sample'] == sample) & (hplot.data.tp == 'TkEle')].tp_sel.unique()


In [ ]:
hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, 'PU0', 'TkEle', 'EGq3Pt15', 'GENEtaBC')


print hsets[0].name

In [ ]:
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

import array

class HWrapper(object):
    def __init__(self, histo):
        self.histo = histo
    
    def get(self, debug=False):
        return self.histo

fake_histoname = uuid.uuid4().hex[:6]

for pu in ['PU0', 'PU200']:
    for gen_sel in ['GENEtaBC']:
        for tp in ['EG', 'TkEleEL']:
            for tp_sel_strings in [('EGq3Pt{}', 'EGq3'), ('EGq4Pt{}', 'EGq4'), ('EGq5Pt{}', 'EGq5')]:
                pt_points = [10, 15, 20, 25, 30, 40]
                pt_95 = []
                pt_95_err = []
                for pt in pt_points:
#                     for (i=0; i<ngr; i++)
#       grint->SetPoint(i, gr->GetX()[i], 0);
        
                    tp_sel = tp_sel_strings[0].format(pt)
                    hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, tp_sel, gen_sel)
                        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        #                 draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])

#                     if pt == 20:
                    tf_yc.SetParameters(0.6, pt, 0.1, 1.0, 0.0)
#                     else:
#                         tf_yc.SetParameters(1, 20, 4, 1.0, 0.0)
                    h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]

                    result = h_eff_vs_pt[0].Fit(tf_yc, 'EMQRS+')

                    grint = ROOT.TGraphErrors(h_eff_vs_pt[0].GetN())
                    stuff.append(grint)
                    grint.SetTitle("Fitted line with .95 conf. band")
                    for point in range(0, h_eff_vs_pt[0].GetN()):
                        grint.SetPoint(point, h_eff_vs_pt[0].GetX()[point], 0)
                    ROOT.TVirtualFitter.GetFitter().GetConfidenceIntervals(grint)

                        
                    print '------- {}'.format( text)
                    print result.Print()
                    pt095 = tf_yc.GetX(0.95)
                    pt_95.append(pt095)

                    prev_err = 0
                    error = 0
                    for idx,point in enumerate(grint.GetX()):
#                         print idx, point, grint.GetEY()[idx]
                        if pt095 > point:
                            prev_err = grint.GetEY()[idx]
                        else:
                            error = (prev_err+grint.GetEY()[idx])/2
                            break
                    pt_95_err.append(error)
                    print '       pt th: {}, 95% eff: {}, err: {}'.format(pt, pt095, error)
                    #                 points.append((tp, pu, pt, pt095))
                    draw(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                    ROOT.gStyle.SetOptFit(11111)
                    grint.Draw('same')

                print len(pt_points)
                print len(pt_95)
                print len(pt_95_err)
                print pt_95_err
                
                graph = ROOT.TGraphErrors(len(pt_points), array.array('d', pt_points), 
                                                          array.array('d', pt_95),
                                                          array.array('d', [0.]*len(pt_points)),
                                                          array.array('d', pt_95_err) )
                graph.SetMarkerStyle(7)
                graph.SetMarkerColor(2)
                stuff.append(graph)

                hplot.data = hplot.data.append({'sample': sample,
                                                'pu': pu,
                                                'tp': tp,
                                                'tp_sel': tp_sel_strings[1],
                                                'gen_sel': gen_sel,
                                                'classtype': fake_histoname,
                                                'histo': HWrapper(graph),},
                                                ignore_index=True)

                
                

In [ ]:
hplot.data[hplot.data.classtype == fake_histoname]


In [ ]:
scaling_params = pd.DataFrame(columns=['tp', 'tp_sel', 'params'])
ROOT.gStyle.SetOptFit(11111)

for pu in ['PU0','PU200']:
    for tp in ['EG', 'TkEleEL']:
        for tp_sel in ['EGq3', 'EGq4', 'EGq5']:
            hsets, labels, text = hplot.get_histo(fake_histoname, sample, pu, tp, tp_sel, ['GENEtaBC'])
            result = hsets[0].Fit('pol1', 'MES+', '', 5, 35)
            result.GetParams()
            scaling_params = scaling_params.append({'pu': pu,
                                                     'tp': tp,
                                                     'tp_sel': tp_sel+'EtaBC',
                                                     'params': (result.GetParams()[0], result.GetParams()[1])},
                                                      ignore_index=True)
            draw(hsets, labels, text=text, min_y=0, max_y=60, y_axis_label='p_{T} @ 95% eff. [GeV] ', x_axis_label='p_{T} thresh. [GeV]')

###### 

In [ ]:
scaling_params

In [ ]:
scaling_params.to_csv(r'scaling_params_{}.csv'.format(version_V9))